In [1]:
import glob
import sys
import numpy as np
np.seterr(divide = 'ignore')

from enterprise.pulsar import Pulsar
from enterprise.signals import signal_base
from enterprise.signals import parameter
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals

import dpdm_delay

# import customized files to introduce 2T modes.
import gp_bases_new as gp_bases
import pl_prior as gp_priors

from enterprise.signals import selections

import multiprocess
from PTMCMCSampler.PTMCMCSampler import PTSampler


You'll need sksparse for get_coefficients() with common signals!
Cannot import PINT? Meh...
Do not have mpi4py package.


In [2]:
datadir = 'DR2'
parfiles = sorted(glob.glob(datadir + '/newpars/*.par'))
timfiles = sorted(glob.glob(datadir + '/toas/*.tim'))

psrs=[]
for ipsr in range(len(parfiles)):
    psr = Pulsar(parfiles[ipsr], timfiles[ipsr])
    psrs.append(psr)
    print(psr.name)

J0437-4715
J0613-0200
J0711-6830


J1017-7156
J1022+1001
J1024-0719
J1045-4509


J1125-6014


J1446-4701


J1545-4550
J1600-3053
J1603-7202
J1643-1224
J1713+0747
J1730-2304
J1732-5049
J1744-1134


J1824-2452A
J1832-0836
J1857+0943
J1909-3744
J1939+2134
J2124-3358
J2129-5721
J2145-0750


J2241-5236


In [3]:
# red noise
nmodes = 30


log10_A = parameter.Constant()
gamma = parameter.Constant()
pl = gp_priors.powerlaw(log10_A=log10_A, gamma=gamma)
dm_basis = gp_bases.createfourierdesignmatrix_dm(nmodes=nmodes)
red_basis = gp_bases.createfourierdesignmatrix_red(nmodes=nmodes)
selection = selections.Selection(selections.by_band)

dmn = gp_signals.BasisGP(pl, dm_basis, name='dm', coefficients=False)
spn = gp_signals.BasisGP(pl, red_basis, name='sp',coefficients=False)
bdn = gp_signals.BasisGP(pl, red_basis, name='bd',coefficients=False,selection=selection)

# white noise
backend = selections.Selection(selections.by_backend)
efac = parameter.Constant()
equad = parameter.Constant()

ef = white_signals.MeasurementNoise(efac=efac,selection=backend)
eq = white_signals.EquadNoise(log10_equad=equad,selection=backend)
wnb = ef + eq

# timing model
tm = gp_signals.TimingModel()

# Bayes ephmeris
eph = deterministic_signals.PhysicalEphemerisSignal(use_epoch_toas=True)


# include best fit parameters
Dict = np.load('noise_params/noisepars_m'+str(nmodes)+'.npy',allow_pickle=True,encoding='bytes').item()

In [4]:
def make_bins(n, nbins=125):
    
        logma_min,logma_max=-23.5 + 2.5 * n / nbins , -23.5 + 2.5*(n+1) / nbins
        log10_ma=parameter.LinearExp(logma_min,logma_max)('x_dp_log10_ma')#n=0~99
        return log10_ma

#def fix_ma(n,nbins=250):
#        logma=-23.5+2.5*float(n)/float(nbins)
#        log10_ma=parameter.Constant(logma)('x_dp_log10_ma')#n=0~99
#        return log10_ma

In [5]:
def single_task(a, b, i, ret, model, test=False):
    
        np.random.seed()
    
        log10_ma = make_bins(i)
        #log10_eps = dpdm_delay.parameter.LinearExp(-28,-16)('x_dp_log10_eps')
        log10_eps = parameter.Uniform(-28.0, -16.0)('x_dp_log10_eps')
        dp = dpdm_delay.dpdm_block(model=model,log10_ma=log10_ma)

        model0  = tm + wnb + dmn + spn + eph + dp
        model1  = tm + wnb + dmn + spn + eph + dp + bdn

        signal=[]
        for psr in psrs:
            if psr.name in ['J0437-4715','J1939+2134']:
                signal.append( model1( psr ) )
            else:
                signal.append( model0( psr ) )

        pta = signal_base.PTA(signal)
        pta.set_default_params(Dict)
        if test == True:
            return pta

        outDir="bayes/"+model+"_m"+str(nmodes)+"/n"+str(i).zfill(3)+'/'
#============================
        # to make the initial values
        xs = {par.name:par.sample() for par in pta.params}
        for parname in Dict.keys():
            if parname in xs.keys():
                xs[parname] = Dict[parname]
        
        x0 = np.hstack([xs[key] for key in sorted(xs.keys())])    
        ndim = len(x0)
#============================
        x0[-19:-8] = 0.
        x0[-8] = -25.
        
        groups=[]
        groups.append(range(ndim-8,ndim))
        groups.append(range(ndim))
        
        print(pta.get_lnlikelihood(x0))
        
        
        covdiag = np.ones(ndim)*0.01

        sampler = PTSampler(ndim, pta.get_lnlikelihood, pta.get_lnprior,
                        cov=np.diag(covdiag),outDir=outDir,groups=groups,resume=False)

        sampler.sample(x0, 60000, isave=1000)
        
        chain=np.loadtxt(outDir+'/chain_1.txt')

        logep=chain[int(0.5*len(chain)):,-12]

        ret.value = np.percentile(logep,95)


In [ ]:
model="U1B_corr"

for j in range(7):
    a = 0 + j*12
    b = min(0 + (j+1)* 12,125)

    print(a,b)

    jobs = []
    RETs={}
    for i in range(a,b):        
        RETs[i] = multiprocess.Manager().Value('i',0)
        p = multiprocess.Process(target=single_task, args=(a,b,i,RETs[i],model))
        jobs.append(p)
        p.start()
    for p in jobs:
        p.join()
    limits = [RET.value for RET in RETs.values()]


#word = '_'.join(('bayes_limit/'+model,'m'+str(nmodes),str(a).zfill(3),str(b).zfill(3))) 
#np.save(word,limits)

0 12
model:U1B, ***correlated**
model:U1B, ***correlated**
model:U1B, ***correlated**
model:U1B, ***correlated**
model:U1B, ***correlated**
model:U1B, ***correlated**
model:U1B, ***correlated**
model:U1B, ***correlated**
model:U1B, ***correlated**
model:U1B, ***correlated**
model:U1B, ***correlated**
model:U1B, ***correlated**
